In [ ]:
pip uninstall matplotlib

In [ ]:
pip install --upgrade matplotlib

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data_tamanho = 2000
dataset = pd.read_csv("predit_dados_nov_abr_correto_2.csv")
dataset.drop(columns =["Date"], inplace = True) 
dataset = dataset[:data_tamanho] # definir o tamanho da amostra tamanho_amostra 

ImportError: DLL load failed: Não foi possível encontrar o módulo especificado.

In [4]:
# multivariate multi-step mlp example
import functools
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from matplotlib import pyplot
from keras.optimizers import Adam 
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.regularizers import l2
from keras.regularizers import l1
#import sys
import numpy
#numpy.set_printoptions(threshold=sys.maxsize)

#definie os limiares para a padrinização
std_min = 0.1
std_max = 0.9

#Determina um limiar de defasagem para os valores minimos para casos onde os valores sejam diferentes do que o esperado
min_lag=1
max_lag=1
min_values = []
max_values = []

new_sheet = None  

# choose a number of time steps
n_steps_in, n_steps_out = 10, 8
quant_colunas = 6

def normalize_equation(outliers,elem):
    return np.multiply(np.divide(np.subtract(elem,outliers[0]),np.subtract(outliers[1],outliers[0])),(std_max-std_min))+std_min

def normalize_(outliers, array,is_numpy=False):
    if(not is_numpy):
        array.rename_axis(None, inplace=True)
    min = outliers[0]
    max = outliers[1]
    if(is_numpy):
        rValue = np.apply_along_axis(functools.partial(normalize_equation,(min,max)),0,array)
    else:    
        rValue = array.apply(functools.partial(normalize_equation,(min,max)))
    return rValue

def normalize(array,is_numpy_=False):
    min = array.min().min()*min_lag
    max = array.max().max()*max_lag
    return normalize_((min,max),array,is_numpy_),min,max

def normalize_ext_parameter(array, min_ext, max_ext, is_numpy_= False):
    min = min_ext
    max = max_ext
    return normalize_((min,max),array,is_numpy_),min,max

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


for name in range(0,quant_colunas):
    new_elem_sheet,new_min,new_max = normalize(dataset.iloc[:, name])
    min_values.append(new_min)
    max_values.append(new_max)
    if(new_sheet is None):
        new_sheet = new_elem_sheet
    else:
        new_sheet = pd.concat([new_sheet, new_elem_sheet], axis=1,sort=False)
print("min_values normalize %s" %name + ": %s" %min_values + "maxn_values normalize %s" %name + ": %s" %max_values)


#Prepara banco de dados, recebendo cada entrada em um vetor
input0 = new_sheet.iloc[:,0].values
input0 = input0.reshape((len(input0), 1))
input1 = new_sheet.iloc[:,1].values
input1 = input1.reshape((len(input1), 1))
input2 = new_sheet.iloc[:,2].values
input2 = input2.reshape((len(input2), 1))
input3 = new_sheet.iloc[:,3].values
input3 = input3.reshape((len(input3), 1))
input4 = new_sheet.iloc[:,4].values
input4 = input4.reshape((len(input4), 1))
input5 = new_sheet.iloc[:,5].values
input5 = input5.reshape((len(input5), 1))

entrada = hstack((input0, input1, input2, input3, input4, input5)) #junta o banco de dados

X, y = split_sequences(entrada, n_steps_in, n_steps_out)

# flatten input: Transforma cada entrada em uma linha, pois estão todas separadas em no número de steps da quantidade de defasagens 
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

new = []
new_size = (len(X), len(X[1]))
new = numpy.random.uniform(low=0.0, high=0.0, size=new_size)
tam = (len(entrada[1])-1) # quantidade de inputs
count = 0
for a in range(tam): # coloca as entradas em ordem    
    b = a
    for c in range(n_steps_in):
        new[:,count] = X[:,b]
        b = (b+tam)
        count = count+1



#n_input = testX.shape[1] * testX.shape[2]
#testX = testX.reshape((testX.shape[0], n_input))

# Separa dataset em trainamento e teste
trainX, testX, trainy, testy= train_test_split(new, y, test_size=0.1)

#opt = Adam(lr=0.01, decay=0.01)
opt = Adam(lr=0.001)

# define model
model = Sequential()
model.add(Dense(100, activation='hard_sigmoid', input_dim=n_input,  kernel_initializer=glorot_uniform(seed=23))) # kernel_regularizer= l2(0.1)
#model.add(Dense(90, activation='sigmoid', kernel_initializer=glorot_uniform(seed=1))) # kernel_regularizer= l2(0.1)
model.add(Dense(n_steps_out, activation='linear',  kernel_initializer=glorot_uniform(seed=23))) # como é previsão usa saída com função de ativação linear, que é a default
model.compile(optimizer=opt, loss = 'MSE', metrics = ['mean_absolute_percentage_error','MSE'] )
# fit model
#model.fit(X, y, epochs=2000, verbose=0)

# patient early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
# fit model
#history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=40000, callbacks=[es], batch_size=20)
model.fit(trainX, trainy, epochs=10, batch_size=20)
#
## evaluate the model
#_, train_acc = model.evaluate(trainX, trainy, verbose=0) #usado para testar a acurácia do modelo
#_, test_acc = model.evaluate(testX, testy, verbose=0)
#print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
## plot loss learning curves
#pyplot.subplot(211)
#pyplot.title('MSE', pad=-40)
#pyplot.plot(history.history['loss'], label='train')
#pyplot.plot(history.history['val_loss'], label='test')
#pyplot.legend()
## plot accuracy learning curves
#pyplot.subplot(212)
#pyplot.title('Accuracy', pad=-40)
#pyplot.plot(history.history['accuracy'], label='train')
#pyplot.plot(history.history['val_accuracy'], label='test')
#pyplot.legend()
#pyplot.show()
#

min_values normalize 5: [79, 139, 68, 102, 22, 231]maxn_values normalize 5: [1470, 1573, 412, 1198, 112, 2430]
Epoch 1/10
1785/1785 [==============================] - 0s 159us/step - loss: 0.0955 - mean_absolute_percentage_error: 70.6115 - MSE: 0.0955
Epoch 2/10
1785/1785 [==============================] - 0s 79us/step - loss: 0.0089 - mean_absolute_percentage_error: 26.3375 - MSE: 0.0089
Epoch 3/10
1785/1785 [==============================] - 0s 91us/step - loss: 0.0075 - mean_absolute_percentage_error: 24.1947 - MSE: 0.0075
Epoch 4/10
1785/1785 [==============================] - 0s 87us/step - loss: 0.0069 - mean_absolute_percentage_error: 23.0164 - MSE: 0.0069
Epoch 5/10
1785/1785 [==============================] - 0s 78us/step - loss: 0.0066 - mean_absolute_percentage_error: 22.4468 - MSE: 0.0066
Epoch 6/10
1785/1785 [==============================] - 0s 82us/step - loss: 0.0064 - mean_absolute_percentage_error: 21.9706 - MSE: 0.0064
Epoch 7/10
1785/1785 [==========================

In [302]:
x_input = array([76, 76, 77, 80, 82, 83, 83, 85, 82, 80, 183, 180, 191, 190, 190, 200, 197, 197, 197, 196, 138, 133, 143, 158, 169, 170, 165, 154, 141, 136, 90, 92, 93, 92, 92, 93, 95, 95, 96, 96, 48, 48, 46, 45, 45, 44, 43, 43, 42, 42])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0) # usado para saber qual é a resposta, dada uma entrada
print(yhat)

[[1.8317046 1.5912037 1.3293186 1.9649228 2.0902631 1.3865775 1.732712
  1.8319149]]


In [158]:
dataset

array([[ 250,  244,   91,  214,   37,  575],
       [ 191,  305,   97,  208,   36,  643],
       [ 169,  380,  115,  207,   36,  727],
       [ 136,  290,  114,  201,   38,  615],
       [ 131,  286,  100,  196,   38,  605],
       [ 126,  239,   95,  210,   38,  563],
       [ 134,  242,  120,  207,   36,  563],
       [ 165,  298,  118,  203,   35,  626],
       [ 149,  321,  161,  225,   35,  680],
       [ 166,  380,  145,  243,   34,  771],
       [ 142,  290,  153,  270,   36,  700],
       [ 125,  313,  139,  276,   38,  735],
       [ 140,  313,  137,  378,   41,  860],
       [ 184,  321,  158,  339,   44,  823],
       [ 156,  350,  154,  326,   40,  841],
       [ 140,  365,  139,  409,   41,  958],
       [ 134,  311,  135,  339,   42,  812],
       [ 122,  277,  131,  321,   40,  749],
       [ 112,  263,  120,  283,   39,  683],
       [ 105,  235,  105,  256,   40,  617],
       [ 102,  215,   97,  225,   40,  551],
       [ 100,  205,   94,  196,   39,  505],
       [  

In [5]:
trainy[1]

array([0.23060482, 0.23060482, 0.22223738, 0.21095953, 0.20077308,
       0.19968167, 0.21168713, 0.22914961])

In [6]:
[[0.23103653 0.24108489 0.22472858 0.20162311 0.19735977 0.19679828
  0.19337028 0.1927958 ]]

array([0.1713156 , 0.17016535, 0.15521208, 0.13450755, 0.12703091,
       0.12415528, 0.12588066, 0.12645579, 0.13393242, 0.12818116,
       0.18647141, 0.19260809, 0.19595537, 0.19260809, 0.18089261,
       0.16136681, 0.15467225, 0.15299861, 0.15746165, 0.15913529,
       0.31860465, 0.32093023, 0.28837209, 0.22790698, 0.20232558,
       0.2       , 0.21162791, 0.22790698, 0.23023256, 0.24418605,
       0.28978102, 0.28686131, 0.29708029, 0.30729927, 0.29562044,
       0.28029197, 0.27080292, 0.24087591, 0.22992701, 0.22262774,
       0.29555556, 0.28666667, 0.28666667, 0.23333333, 0.22444444,
       0.22444444, 0.21555556, 0.21555556, 0.21555556, 0.21555556])